**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [12]:
!pip install keras-flops

In [19]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras_flops import get_flops

In [20]:
# Глобальные константы
CLASSES = 100
BATCH_SIZE = 128
LEARNING_RATE = 1e-2

In [21]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = keras.datasets.cifar100.load_data()

In [22]:
# Преобразуйте метки классов в one_hot формат
y_train = keras.utils.to_categorical(y_train)
y_val = keras.utils.to_categorical(y_val)

In [23]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [128]:
# Задайте архитектуру модели
model = keras.models.Sequential([
    keras.layers.Conv2D(8, (3, 3), strides=2, input_shape=[32, 32, 3], padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(8, (3, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(350),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(CLASSES, activation='sigmoid')
])

In [129]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6965e6


In [130]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_52 (Conv2D)          (None, 16, 16, 8)         224       
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 8)        32        
 hNormalization)                                                 
                                                                 
 activation_31 (Activation)  (None, 16, 16, 8)         0         
                                                                 
 conv2d_53 (Conv2D)          (None, 14, 14, 8)         584       
                                                                 
 batch_normalization_2 (Batc  (None, 14, 14, 8)        32        
 hNormalization)                                                 
                                                                 
 activation_32 (Activation)  (None, 14, 14, 8)       

In [131]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [132]:
# обучения модели
model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=BATCH_SIZE, use_multiprocessing=True, workers=8, epochs=50)

Epoch 1/50
391/391 [==============================] - 9s 19ms/step - loss: 3.9429 - accuracy: 0.1092 - val_loss: 3.5089 - val_accuracy: 0.1784
Epoch 2/50
391/391 [==============================] - 7s 19ms/step - loss: 3.4030 - accuracy: 0.1913 - val_loss: 3.1744 - val_accuracy: 0.2407
Epoch 3/50
391/391 [==============================] - 7s 18ms/step - loss: 3.1979 - accuracy: 0.2248 - val_loss: 3.0312 - val_accuracy: 0.2646
Epoch 4/50
391/391 [==============================] - 7s 19ms/step - loss: 3.0856 - accuracy: 0.2489 - val_loss: 2.9379 - val_accuracy: 0.2828
Epoch 5/50
391/391 [==============================] - 7s 19ms/step - loss: 2.9836 - accuracy: 0.2673 - val_loss: 2.9168 - val_accuracy: 0.2823
Epoch 6/50
391/391 [==============================] - 7s 19ms/step - loss: 2.9213 - accuracy: 0.2780 - val_loss: 2.8453 - val_accuracy: 0.3022
Epoch 7/50
391/391 [==============================] - 7s 19ms/step - loss: 2.8550 - accuracy: 0.2895 - val_loss: 2.8035 - val_accuracy: 0.3049

In [133]:
model.save('BOBA.hdf5')